In one approach, if we have detailed sentences, we can use a dictionary cross reference to categorize items (people, place, thing, etc.) - then use an LLM to generate relevant topics.

Citation: https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [31]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

df = pd.read_csv("../Kaitlin's Workspace/arasaac_synset_mapping_20251106_130530.csv")
df = df.drop_duplicates(subset=['primary_keyword'])

model = SentenceTransformer("all-MiniLM-L6-v2")

sentence = "a long hallway with a rug on the floor"
sentence_emb = model.encode(sentence, convert_to_tensor=True)

df["keyword_emb"] = df["primary_keyword"].apply(
    lambda x: model.encode(x, convert_to_tensor=True))

df["score"] = df["keyword_emb"].apply(
    lambda emb: util.cos_sim(sentence_emb, emb).item())

df_sorted = df.sort_values("score", ascending=False)
print(df_sorted[["primary_keyword", "score"]])


                  primary_keyword     score
3                          carpet  0.625963
7270                     corridor  0.585937
446                         floor  0.509728
9985           intermediate floor  0.493773
17515                  last floor  0.482547
...                           ...       ...
18328                oreo cookies -0.097604
15672       cod with tomato sauce -0.112397
17533  sunflower seeds breadstick -0.113835
18112                tinfoil barb -0.126530
14393                 tuning fork -0.145110

[7909 rows x 2 columns]


In [32]:
df_sorted = df_sorted.drop('keyword_emb', axis=1)

In [33]:
df_sorted.to_csv('output.csv')